<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/ImageClass/CatRabbit/modelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch ile Kendi Datasetinden Eğitim


Bir çok tutorialda torchvision dataset içinden hızlıca bir dataset çekmek tercih edilir çünkü çok kolayca düzgün bir dataset oluşturmuş olursun.

Biz bugün kendi elimizdeki bir data seti kullanarak bu işlemleri yapacağız.

Bunun için bir bulk downloader uzantısı ile googledan bir çok resmi hızlıca kayıt edebilir ya da elimizdeki hazır bir seti kullanabiliriz. Araştırmalar sırasında video çekip bu videodan bir python scripti ile ekran görüntüleri almak gibi teknikler ile de veri setleri hızlıca oluşturulabilir

ancak data bir miktar düzenlemeye ihtiyaç duyabilir.



# Database yapısı


Oldukça basit bir yapıyı takip edeceğiz
                                      
data/                                           
├─ cat/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             
├─ rabbit/             
│  ├─ image001.jpg             
│  ├─ image002.jpg             
│  ├─ ....             

Bu dosya hem zipli hem de sıkıştırılmamış olarak repoda mevcut. Zipli hali colabda çalışırken wget ile kolayca indirmen için

Colabde isen kolayca indir

In [1]:
!wget https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CatRabbit/data.zip

--2024-07-17 13:15:25--  https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CatRabbit/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/ImageClass/CatRabbit/data.zip [following]
--2024-07-17 13:15:26--  https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/ImageClass/CatRabbit/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74171382 (71M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  70.73M   125MB/s    in 0.6s    

2024-07-17 13:15:27 (125 MB/s) - ‘data.zip’

In [3]:
!unzip -q data.zip

# Define Transforms

Transfromların olay resimlere rastgele değişimler uygulamaktır.  

Bu hem elimizdeki data miktarını arttırmak için güzel bir metot hem de farklı koşullar altında da tanıma yapabilmek için önemli.

Örneğin rastgele döndürmeler eklemek sayesinde eğer kedi baş aşağı durursa da onu tanıyacağız.





In [2]:
import torchvision.transforms as transforms

In [4]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(p=0.5)
])

# Data setimizi aktarmak

Bu noktada bizim data setimiz düzenli olduğu için bir metot kullanacağız ancak düzensiz bir data setin var ise nasıl çalışman gerektiğinin örneğide olacak


Ayrıca bizim data setimizde iki sınıf içinde yakın miktarda görsel var. Bu iyi bir dataset için önemli bir yapıdır.

In [7]:
import torchvision
from torchvision.datasets import ImageFolder

In [9]:
dataset = torchvision.datasets.ImageFolder(root='data', transform=transform)

# Custom Dataset

Eğer dataset dosya yapın bizim örnekteki gibi düzenli değil ise  aşağıdaki methodu uygulayacaksın. Ama düzenli bir datasetine sahip olmak her zaman daha iyidir.


# Split Data


Amacımız test, validation  ve train oluşturmak.


validation -> eğitim sırasında parametre optimizasyonu için kullanılan veri kısmı

test-> model eğitildikten sonra performans ölçümü için kullanılır.

In [10]:
splits = [0.8, 0.1, 0.1]

In [15]:
split_sizes = []
for sp in splits[:-1]:
    split_sizes.append(int(sp * len(dataset)))
split_sizes.append(len(dataset) - sum(split_sizes))

In [17]:
import torch
import torch.utils
train_set, test_set, val_set = torch.utils.data.random_split(dataset, split_sizes)

In [19]:
train_set, test_set, val_set = torch.utils.data.random_split(dataset, split_sizes)

In [20]:
val_set.transform = test_set.transform = transforms.Compose([
    transforms.Resize((128, 128))
])

# DataLoader

In [23]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [24]:
dataloaders = {
    "train": DataLoader(train_set, batch_size=16, shuffle=True),
    "test": DataLoader(test_set, batch_size=16, shuffle=False),
    "val": DataLoader(val_set, batch_size=16, shuffle=False)
}

# Define the Classifier

In [25]:
class CustomModel(torch.nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        self.linear1 = torch.nn.Linear(128, 256)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(256, 3)
        self.softmax = torch.nn.Softmax()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

In [26]:
class CustomModel(torch.nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        self.linear1 = torch.nn.Linear(128, 256)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(256, 1)
        self.softmax = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x